In [ ]:
import warnings
import matplotlib.pyplot as plt
import numpy as np
from pylj import md, util, sample, forcefields

warnings.filterwarnings('ignore')

# Atomistic simulation

The use of computers in chemistry is becoming more common as computers are increasing in power and the algorithms are becoming more accurate and efficient. Computer simulation of atomic and molecular species takes two "flavours":

- **Quantum calculations**: where approximations are made to the Schrödinger equation to allow for its calculation for a multi-electron system (more information about this can be found in Atkins & Friedman's *Molecular Quantum Mechanics*),
- **Classical simulations**: where model interactions between atoms are developed to allow chemical systems to be simulated,

This exercise will focus on the latter.

## Classical simulation

One of the most popular models used to simulate interparticle interactions is known as the **Lennard-Jones** function. This models the London dispersion interactions between atoms. Hopefully the London dispersion interactions are familiar as consisting of: 

- **van der Waals attraction**: the attraction between atoms that occurs as a result of the formation of instantenous dipole formation,
- **Pauli's exclusion principle**: the repulsion which stops atoms overlapping as no two electrons can have the same quantum state. 

This means that the Lennard-Jones function is attractive when particles are close enough to induce dipole formation but **very** repulsive when the particles are too close. The Lennard-Jones function has the following form,

$$ E(r) = \frac{A}{r^{12}} - \frac{B}{r^6}, $$

where $E(r)$ is the potential energy at a given distance $r$, while $A$ and $B$ are parameters specific to the nature of the given interaction. In the cell below, write a function to calculate the energy of an interaction between two argon atoms for given range of distances using the Lennard-Jones function as the model.

In [ ]:
def lennard_jones(A, B, r):
    return ◽◽◽

Then use this function to plot the potential energy from $r=3$ Å to $r=8$ Å, and discuss the shape and sign of this function in terms of the attractive and repulsive regimes, when the values of ε and σ are as follows.

| A/$\times10^{-134}$Jm$^{12}$ | B/$\times10^{-78}$Jm$^6$ |
|:------:|:------:|
| 1.36 | 9.27 |

In [ ]:
r = np.linspace( ◽◽◽, ◽◽◽, ◽◽◽ )
E = ◽◽◽
plt.plot( ◽◽◽, ◽◽◽ )
plt.xlabel( ◽◽◽ )
plt.ylabel( ◽◽◽ )
plt.show()

We are going to use the software pylj. This is a Python library, which means it is a collection of functions to enable the simulation of molecular dynamics. An example of a function in pylj is the `pairwise.lennard_jones_energy`, which can be used similar to the `lennard_jones` function that you have defined above.

In [ ]:
r = np.linspace( 3e-10, 8e-10, 100 )
E = forcefields.lennard_jones(r, [1.36e-134, 9.27e-78])

plt.plot( r, E )
plt.xlabel( '$r/m$' )
plt.ylabel( '$E/J$' )
plt.show()

In addition to the functions associated directly with the molecular dynamics simulations, pylj is also useful for the visualisation of simulations (an example of a visualiation environment in pylj is shown below).

<img src="fig1.png" width="500px">
*Figure 1. The Interactions sampling class in pylj.*

In this exercise, we will use pylj to help to build a molecular dynamics simulation where the particles interact via a Lennard-Jones potential. 

## Molecular dynamics 

Molecular dynamics or MD is the process of using Newton's equations of motion to probe the dynamical nature of a given system, the system that we are interested in is the interaction of argon atoms through a Lennard-Jones potential. The algorithm that is used in molecular dynamics simulations is as follows:

1. initialise the system,
2. start the clock, 
3. calculate the forces of each particle,
4. integrate Newton's equations of motion to step forward in time,
5. sample the system,
6. go to 3.

This process continues for as long as the scientist is interested in.

### Initialisation 

Lets try and use pylj to initialise our system, to do this we use the function `md.initialise`. This function takes 4 inputs:
- number of particles, 
- temperature of the simulation, 
- size of the simulation cell,
- how the particles should be distributed.

The first line is to set up the visualisation environment within the Jupyter notebook, and the third line then sets and plots the particular environment that you want (in this case the `JustCell` environment).

In [ ]:
simulation = md.initialise(16, 300, 50, 'square')
sample_system = sample.JustCell(simulation)

In the above example, a `square` distribution of particles is assigned, however, it is also possible to distribute the particles randomly through the cell (with the keyword `random`). Try this out in the cell above and comment below on why this might cause issues for the molecular dynamics simulation. 

Another important aspect of the initialisation process is to assign the initial velocities for each of the particles. This is achieved using the following equation, 

$$ \mathbf{v}_i = (n_i - \bar{n}) \sqrt{\frac{2T}{\sum_i^N n^2}}, $$

where $n_i$ is a series of numbers of length $N$, where $N$ is the number of particles, drawn from a uniform distribution between 0 and 1, $\bar{n}$ is the mean $n_i$, and $T$ is the initial temperature of the system. The range of values for the velocities can be found for the system above using the following command.

In [ ]:
print(simulation.particles['xvelocity']) # m/s
print(simulation.particles['yvelocity']) # m/s

Having given each particle an initial position and velocity, the system is initialised. 

### Calculate forces

The next stage is to calculate the forces on each of the particles. The forces on each of the atoms can be force directly from the energy (which is given by the Lennard-Jones function discussed above), as the force is the negative of the first derivative of the energy with respect to the distance, 

$$ \mathbf{f} = -\frac{\partial E}{\partial r}. $$

The force of a given interaction can be found using the `forcefields.lennard(dr, constants, force=True)` function available in pylj, below plot the force for the interaction between two argon atoms as was completed for the energy above. 

In [ ]:
r = ◽◽◽
f = ◽◽◽

plt.plot( ◽◽◽, ◽◽◽ )
plt.xlabel( ◽◽◽ )
plt.ylabel( ◽◽◽ )
plt.show()

The knowledge of the forces is then converted to information about the acceleration on each particle using Newton's second law, 

$$ \mathbf{f} = m\mathbf{a}, $$

where $m$ is the mass of the particle. 

It is possible to calculate the forces on all of the particles within a system in pylj using the `compute_force` function, as shown below. 

In [ ]:
simulation.compute_force()
print(simulation.particles['xacceleration']) # m/s2
print(simulation.particles['yacceleration']) # m/s2

### Integration

With knowledge of the particle positions, velocities and accelerations it is now possible to make use of Newton's equations of motion to move forward in time. This is achieved using an *integrator*, which integrates Newton's equations of motion. A [wide variety](https://en.wikipedia.org/wiki/Molecular_dynamics#Integrators) of MD integrators exist, but we will discuss one of the simplest, known as the Verlet integrator. This has the form,

$$ \mathbf{x}_1 = \mathbf{x}_0 + \mathbf{v}_0\Delta t + \frac{1}{2}\mathbf{a}_0 \Delta t^2, $$

for the first step and, 

$$ \mathbf{x}_{n+1} = 2\mathbf{x}_n - \mathbf{x}_{n-1} + \mathbf{a}_0 \Delta t^2, $$

for subsequent steps. Below, define a function to perform the integration in either the *x* or *y* dimension using the Verlet integrator. 

In [ ]:
def verlet(position, previous_position, velocity, acceleration, timestep, i):
    if i == 0:
        ◽◽◽
    else:
        ◽◽◽        

### Sampling

Following the integration, the next stage is to sample the system. Currently we are only interested in plotting the particles positions, this can be achieved using the `update` function (assuming that the sampling environment has been defined). 

In [ ]:
simulation = md.initialise(16, 300, 50, 'square')
sample_system = sample.JustCell(simulation)

In [ ]:
simulation.compute_force()
◽◽◽ = verlet(◽◽◽, ◽◽◽, ◽◽◽, ◽◽◽, ◽◽◽, ◽◽◽)
sample_system.update(simulation)

It might be hard to see the change, try making the system update on a loop (essentially carrying out step 6 of the algorithm) and see is the MD simulation is working (it is probably best to only update the plot occasionally, as this is the slowest part of the method, consider how this may be done).

In [ ]:
# Cell for building MD simulation. 